Following your advice, during the debug process I will focus on the case with single population of 10_000 items. If the results of training will be good in this case then I will expand to the case with mixed population.

The first things I've decided to explore is the **learning curves** when **sigmoid** activation is applied on all layers. The dataset has **100_000 rows**, train-validation split is **0.8**. 

* The **input** is fraction of requests in previous 4 time slots transformed by $ log(10^{-5} + frac) $, the **output** is unchanged.
* learning rate: 0.1
* number of iterations: 1000
* stochastic gradient descent

<img src="1_err_curves.png"/>

It looks like the model did not learned much after the first iteration and the validation error is 2 orders of magnitude higher than the optimal. Let's see what predictions this model makes.

<img src="2_order_plot.png"/>

The model learnt that the popularity is exaclty the same (except item #1, for which the popularity is a small bit lower).<br\>
Let's see how the optimal predictor, average of previous time slots, behaves.

<img src="3_order_plot.png"/>

Let's confirm that the torch implementation behaves the same, only this time I will limit the number of iterations to 200, since it seems that in previous case the learning finished much earlier.

| Learning curves | Ordering |
| --- | --- |
|<img src="4_err_curves.png"/>|<img src="4_order_plot.png"/>|

The final error is comparable in both cases: $1.5 * 10^{-7}$ and $2.3 * 10^{-7}$. The predicted popularities are also comparable, but in torch case there is a small deviation from the flat line.

Now I decided to repeat the same experiment for neural networks with linear activations. Since with such high learning rate the learning was divirging, I reduced the learning rate to 0.001 and the number of iterations back to 1000.<br/>
After the first batch the error was much larger than after all the following, so I removed it from the plot.

| Learning curves | Ordering |
| --- | --- |
| My NN <img src="5_err_curves.png"/> | My NN <img src="5_order_plot.png"/> |
| Torch <img src="6_err_curves.png"/> | Torch <img src="6_order_plot.png"/> |

The predicted popularities follow the real popularities closer in this case, but for some items the popularity is predicted to be negative. The error is the same as in the case of sigmoid activation.

Now let's see how the error behaves when the size of the population is reduced.<br/>
In this case I've limited the number of iterations to 100 for 500-10000 population size range and to 1000 iterations in range 5-100. The learning rates are the same as in previous cases.

| 500 - 10000 | 5 - 100 |
| --- | --- |
|My NN (sigmoids) <img src="7/1_mulp_pop_err_plot.png"/> | My NN (sigmoids) <img src="7/2_mulp_pop_err_plot.png"/>|
|Torch (sigmoids) <img src="7/3_mulp_pop_err_plot.png"/> | Torch (sigmoids) <img src="7/4_mulp_pop_err_plot.png"/>|
|My NN (linear) <img src="7/5_mulp_pop_err_plot.png"/> | My NN (linear) <img src="7/6_mulp_pop_err_plot.png"/>|
|Torch (linear) <img src="7/7_mulp_pop_err_plot.png"/> | Torch (linear) <img src="7/8_mulp_pop_err_plot.png"/>|

Independent of population size, training and validation errors stay at 3 orders of magnitude higher from optimal. Further investigation is needed.<br/>
For now let's investigate how other activation functions or loss functions behave.

At first I've decided to test the ReLU activation function. The Torch implementation initializes all of the weigths positive. Since the input is negative numbers ($log(x), 0 < x < 1)$), the Torch implementation was not learning, so I changed the ReLU to LeakyReLU, which is $0.01 * x, x < 0$, for both implementations for them to be comparable. Here are the learning curves and ordering.

| Learning curves | Ordering |
| --- | --- |
| My NN <img src="8/1_err_curves.png"/> | My NN <img src="8/1_order_plot.png"/> |
| Torch <img src="8/2_err_curves.png"/> | Torch <img src="8/2_order_plot.png"/> |

In this 2 cases the error was smaller than in previous, 1.5 orders of magnitude higher than optimal. Both cases are good at predictiong popularities for most popular contents, especially Torch implementation, but since the activation function is LeakyReLU there are negative predictions.

I had troubles implementing KL-divergence, so I decided to do last experiment - also transform the output with the same rule: $ log(10^{-5} + frac) $. Then I trained the simple linear NN, without activation. Here are the results:

| Learning curves | Ordering |
| --- | --- |
| My NN <img src="9/1_err_curves.png"/> | My NN <img src="9/1_order_plot.png"/> |
| Torch <img src="9/2_err_curves.png"/> | Torch <img src="9/2_order_plot.png"/> |

In this case the NN perfectly learned the correct ordering ang popularities even after 10 iterations. The training and validations error are shown better than optimal, but it is because average of log of popularity is not equivalent to average of popularity and thus is not guaranteed to be optimal.